# **Stanford 3D Indoor Scene Dataset (S3DIS)**

This notebook Visualizes the [S3DIS](https://svl.stanford.edu/assets/papers/3D_Semantic_Parsing.pdf). How it is structured and how it is segmented, and how it is partitioned into small blocks.

Install and import libraries

In [1]:
# !pip install open3d==0.16.0 # must be at least 0.16.0

In [1]:
import os
import re
from glob import glob
import time
import numpy as np
import pandas as pd
import open3d as o3
# from open3d import JVisualizer # For Colab Visualization
from open3d.web_visualizer import draw # for non Colab
from vis_config import CATEGORIES, COLOR_MAP
import matplotlib.pyplot as plt
%matplotlib inline


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [2]:
import importlib
import utils
importlib.reload(utils)
from utils import get_space_data, get_slice, get_partitions

Get Dataset path and save path for new dataset

In [3]:
ROOT = r'D:\Datasets\Stanford3dDataset_v1.2_Aligned_Version'

Helpers for categories and colors

## Visualize single room

In [5]:
import numpy as np
import open3d as o3

# Read the data
point_cloud = np.loadtxt(ROOT + "\\Area_1\\conferenceRoom_1\\conferenceRoom_1.txt")
points = point_cloud[:, :3]  # x, y, z
colors = point_cloud[:, 3:] / 255  # RGB, normalized

# Create a PointCloud object
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(points)
pcd.colors = o3.utility.Vector3dVector(colors)

# Create an axis (coordinate frame)
# axis = o3.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0]) # add axis

# Visualize the point cloud and axis
# o3.visualization.draw_geometries([pcd, axis]) # Use this if using axis
o3.visualization.draw_geometries([pcd]) # Use this if not using axis

## Visualize multiple rooms from one area

In [8]:
# Load the first point cloud
point_cloud1 = np.loadtxt(ROOT + "\\Area_1\\conferenceRoom_1\\conferenceRoom_1.txt")
points1 = point_cloud1[:, :3]  # x, y, z
colors1 = point_cloud1[:, 3:] / 255  # RGB, normalized

# Load the second point cloud
point_cloud2 = np.loadtxt(ROOT + "\\Area_1\\conferenceRoom_2\\conferenceRoom_2.txt")
points2 = point_cloud2[:, :3]  # x, y, z
colors2 = point_cloud2[:, 3:] / 255  # RGB, normalized

point_cloud3 = np.loadtxt(ROOT + "\\Area_1\\copyRoom_1\\copyRoom_1.txt")
points3 = point_cloud3[:, :3]  # x, y, z
colors3 = point_cloud3[:, 3:] / 255  # RGB, normalized

# Concatenate the points and colors
points_combined = np.concatenate((points1, points2, points3))
colors_combined = np.concatenate((colors1, colors2, colors3))

# Create a PointCloud object
pcd_combined = o3.geometry.PointCloud()
pcd_combined.points = o3.utility.Vector3dVector(points_combined)
pcd_combined.colors = o3.utility.Vector3dVector(colors_combined)

# Visualize the combined point cloud
o3.visualization.draw_geometries([pcd_combined])


# Visualize one object from a room

In [9]:
# Read the data
point_cloud = np.loadtxt(ROOT + "\\Area_1\\conferenceRoom_1\\Annotations\\door_1.txt")
points = point_cloud[:, :3]  # x, y, z
colors = point_cloud[:, 3:] / 255  # RGB, normalized

# Create a PointCloud object
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(points)
pcd.colors = o3.utility.Vector3dVector(colors)

# Create an axis (coordinate frame)
# axis = o3.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0]) # add axis

# Visualize the point cloud and axis
# o3.visualization.draw_geometries([pcd, axis]) # Use this if using axis
o3.visualization.draw_geometries([pcd]) # Use this if not using axis

## Visualize multiple objects from one room

In [11]:
# Load the first point cloud
point_cloud1 = np.loadtxt(ROOT + "\\Area_1\\conferenceRoom_1\\Annotations\\door_1.txt")
points1 = point_cloud1[:, :3]  # x, y, z
colors1 = point_cloud1[:, 3:] / 255  # RGB, normalized

# Load the second point cloud
point_cloud2 = np.loadtxt(ROOT + "\\Area_1\\conferenceRoom_1\\Annotations\\chair_5.txt")
points2 = point_cloud2[:, :3]  # x, y, z
colors2 = point_cloud2[:, 3:] / 255  # RGB, normalized

point_cloud3 = np.loadtxt(ROOT + "\\Area_1\\conferenceRoom_1\\Annotations\\board_1.txt")
points3 = point_cloud3[:, :3]  # x, y, z
colors3 = point_cloud3[:, 3:] / 255  # RGB, normalized

# Concatenate the points and colors
points_combined = np.concatenate((points1, points2, points3))
colors_combined = np.concatenate((colors1, colors2, colors3))

# Create a PointCloud object
pcd_combined = o3.geometry.PointCloud()
pcd_combined.points = o3.utility.Vector3dVector(points_combined)
pcd_combined.colors = o3.utility.Vector3dVector(colors_combined)

# Visualize the combined point cloud
o3.visualization.draw_geometries([pcd_combined])

# Replacing RGB with semantic labels

| Category | Code | Color | RGB |
|:--------:|:----:|:-----:|:---:|
| ceiling  | 0    | darkslategray | (47, 79, 79) |
| floor    | 1    | saddlebrown | (139, 69, 19) |
| wall     | 2    | forestgreen | (34, 139, 34) |
| beam     | 3    | indigo | (75, 0, 130) |
| column   | 4    | red | (255, 0, 0) |
| window   | 5    | yellow | (255, 255, 0) |
| door     | 6    | lime | (0, 255, 0) |
| table    | 7    | aqua | (0, 255, 255) |
| chair    | 8    | blue | (0, 0, 255) |
| sofa     | 9    | fuchsia | (255, 0, 255) |
| bookcase | 10   | palegoldenrod | (238, 232, 170) |
| board    | 11   | cornflower | (100, 149, 237) |
| stairs   | 12   | hotpink | (255, 105, 180) |
| clutter  | 13   | black | (0, 0, 0) |

In [12]:
area_nums = '1-6' # decide on the number of areas to obtain
area_dict = {}

# get areas based on split
areas = glob(os.path.join(ROOT, f'Area_[{area_nums}]*'))

for area in areas:
    # get all subfolders in area (corresponds to disjoint spaces (or locations))
    spaces = next(os.walk(area))[1]

    # get dict to store spaces
    space_dict = {}

    # for each space
    for space in spaces:
        space = os.path.join(area, space)
        annotations = os.path.join(space, 'Annotations')

        # get individual segmentation filepaths
        segments = glob(os.path.join(annotations, '*.txt'))
        
        # update space dict
        space_dict.update({space.split('\\')[-1] : segments})

    # update area dict
    area_dict.update({area.split('\\')[-1] : space_dict})
        

In [13]:
area_dict['Area_1']['conferenceRoom_1'][0]

'D:\\Datasets\\Stanford3dDataset_v1.2_Aligned_Version\\Area_1\\conferenceRoom_1\\Annotations\\beam_1.txt'

In [14]:
space_data = get_space_data(area_dict['Area_1']['conferenceRoom_1']) # replaces the rgb in each point with the corresponding semantic label (0-13) using the name of each object.
space_data, space_data.shape

(array([[-15.609,  39.505,   2.214,   3.   ],
        [-15.634,  39.518,   2.198,   3.   ],
        [-15.622,  39.514,   2.195,   3.   ],
        ...,
        [-15.339,  39.561,   0.355,   2.   ],
        [-15.331,  39.543,   0.419,   2.   ],
        [-15.351,  39.569,   0.299,   2.   ]], dtype=float32),
 (1136617, 4))

In [16]:
map_colors = lambda x : COLOR_MAP[x]
v_map_colors = np.vectorize(map_colors) # this will take in an array and apply the map_colors function to each element

In [17]:
np.vstack(v_map_colors(space_data[:, 3])).shape # (3,N) Color values for each point

(3, 1136617)

In [18]:
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(space_data[:, :3])
pcd.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(space_data[:, 3])).T/255)
o3.visualization.draw_geometries([pcd])

# Partitioning

In [19]:
space_data = get_space_data(area_dict['Area_1']['conferenceRoom_1']) # replaces the rgb in each point with the corresponding semantic label (0-13) using the name of each object.
# obtain x, y partitions
xyz = space_data[:, :3]

# get 0 min shifted points
xyz_s = xyz - xyz.min(axis=0)
x_parts, y_parts = get_partitions(xyz, xyz_s, c=1.5)

print(x_parts.shape, y_parts.shape)


x_part = x_parts[3]
y_part = y_parts[1]
space_slice = get_slice(space_data, xyz_s, x_part, y_part)
space_slice.shape, space_data.shape, type(space_slice), type(space_data)

(4, 2) (3, 2)


((103769, 4), (1136617, 4), numpy.ndarray, numpy.ndarray)

Note here that the number of points in `space_data` is `1136617` and in the `space_slice` is `103769`

```python
1136617 / 103769 = 10.95
```



In [23]:
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(space_slice[:, :3])
pcd.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(space_slice[:, 3])).T/255)
o3.visualization.draw_geometries([pcd])

# Joining partitions
We can see that when we join the partitions, it is the same as the original space.

In [24]:
sapce_slices_combined_list = []
for xpart in x_parts:
    for ypart in y_parts:
        space_slice = get_slice(space_data, xyz_s, xpart, ypart)
        sapce_slices_combined_list.append(space_slice)
        
sapce_slices_combined = np.vstack(sapce_slices_combined_list)


In [25]:
pcd = o3.geometry.PointCloud()
pcd.points = o3.utility.Vector3dVector(sapce_slices_combined[:, :3])
pcd.colors = o3.utility.Vector3dVector(np.vstack(v_map_colors(sapce_slices_combined[:, 3])).T/255)
o3.visualization.draw_geometries([pcd])